<div style="text-align: center; font-weight: bold; font-size: 300%">MAP 412 - PC 3</div>                                                        <br />      
<div style="text-align: center; font-size: 150%">École Polytechnique</div><br />  
<div style="text-align: center; font-size: 120%">Paul Calot--Plaetevoet</div>

In [1]:
import numpy as np

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import row, gridplot, column

output_notebook(hide_banner=True)

# Méthode des différences divisées

**Implémenter une fonction qui prend en entrée des abscisses $x_k$ et des valeurs $f(x_k)$, et qui retourne les coefficients $f[x_0,\ldots,x_k]$. On pourra utiliser :**

$$
f[x_0,\ldots,x_{n+1}]=\frac{f[x_{1},\ldots,x_{n+1}]-f[x_0,\ldots,x_{n}]}{x_{n+1}-x_0}.
$$

**en remarquant que $f[x_k]=f(x_k)$ pour tout $k$.**

On considère un tableau où à la ligne i, colonne j, se trouve le terme $f[x_{j-i},\ldots,x_{j}]$ noté $\alpha_{i,j}$. Les termes en dessous de la diagonale (i>j) étant donc nuls. On a alors pour $j \geq i>0$ (on numérote à partir de 0): $$ \alpha_{ij} = \frac{\alpha_{i-1,j}-\alpha_{i-1,j-1}}{x_j - x_{j-i}}$$.

In [2]:
def compute_divided_diff_coef(x, y):
    n=len(x)
    L=np.zeros((n,n))
    for k in range(n):
        L[0][k]=y[k]  # on met sur la première ligne toutes les valeurs des points d'interpolation. 
        
    # on construit d'abord les lignes
    for i in range(1,n):
        for j in range(i,n):
            # Ici on calcule f[xj-i,...,xj]
            L[i][j]=(L[i-1][j]-L[i-1][j-1])/(x[j]-x[j-i])
    
    # on ne sélectionne alors que les termes de la diagonale pour obtenir les Cnk.
    coef=np.zeros(n)
    for k in range(n):
        coef[k]=L[k][k]
        
    return coef

**Implémenter une fonction qui prend en entrée des abscisses $x_k$ les coefficients $f[x_0,\ldots,x_k]$ et un vecteurs de points de $[a,b]$, et qui renvoie la valeur de $P_n(f)$ en ces différents points. On pourra utiliser :**

$$
P_n(f)(x)=\sum_{k=0}^n f[x_0,\ldots,x_k] \Pi_k(x),\quad \forall~x\in[a,b].
$$

**et un algorithme d'évaluation de type Horner.**

In [3]:
def poly_interp(coef, xk, x):
    N=len(x)
    n=len(xk)
    p=np.zeros(N)
    
    for m in range(N):
        for k in range(n):
            p[m]=coef[n-k-1]+(x[m]-xk[n-k-1])*p[m] # algorithme d'Horner
            
    return p

**Générer aléatoirement des abscisses $x_k\in[-1,1]$ et des valeurs $y_k\in\mathbb{R}$, $0\leq k\leq 11$, et utiliser les fonctions construites aux deux questions précédentes pour tracer l'unique polynôme $P_n$ de degré $n$ vérifiant $P_n(x_k)=y_k$ pour tout $k$. Commenter.**

In [4]:
n = 10
xk = np.linspace(-1, 1, n+1)
yk = 11*(np.random.rand(n+1))

# vecteurs des points auxquels p sera évalué pour le tracé
a = -1
b = 1
x = np.linspace(a,b,1000)

# calcule des évaluations du polynôme interpolateur en ce points
coef = compute_divided_diff_coef(xk, yk)
p = poly_interp(coef, xk, x)

# Tracer les points (xk, yk) et le polynome p
fig1 = figure(width=490, height=300,title="Tracé du polynôme Pn par la méthode des différences divisées")
fig1.xaxis.axis_label="x"
fig1.yaxis.axis_label="Pn(x)"
fig1.x(xk,yk,color="red")
fig1.line(x,p,color="blue")

fig1.legend.location = "top_left"
show(row(fig1))

C:\Users\Paul\Anaconda3\lib\site-packages\bokeh\models\plots.py:725: UserWarning: 
You are attemptings to set `plot.legend.location` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with the 'legend' parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)


On remarque que le polynôme prend des valeurs élevées (environ 30 en valeur absolue) proches des bornes de l'intervalle considéré. Cela peut être dû aux erreurs en machine qui créént une divergence du polynôme comme nous avons pu le voir en amphi.

# Phénomène de Runge

**Considérer la fonction :**

$$
f(x) =\frac{1}{0.1 +x^2}
$$

**sur l'intervalle $[-1,1]$, et tracer le polynôme d'interpolation $P_n(f)$ pour différentes valeurs de $n$ et différents choix de $x_k$, ainsi que l'erreur entre $f$ et $P_n(f)$. Commenter.**

In [5]:
def f(x):
    return 1/(0.1+x*x)

In [6]:
# En utilisant les fonctions de la partie précedente, tracer les points (xk, yk) et le polynome p 
# pour différentes valeurs de n et différents choix de xk (uniforme et Chebyshev).
# Tracer également l'erreur entre f et p.

n = 10
a = -1
b = 1
x = np.linspace(a,b,1000)


# ----------------------- pour xk uniformément réparti ---------------------------------------
     # définition des vecteurs et utilisation des fonctions
xk = np.linspace(-1, 1, n+1)
yk = f(xk)
coef = compute_divided_diff_coef(xk, yk)
p = poly_interp(coef, xk, x)

    # tracés
    
            # polynôme d'interpolation
fig1 = figure(width=490, height=300,title="Méthode des différences divisées - répartition uniforme ")
fig1.x(xk,yk,color="red",legend="points d'interpolation")
fig1.line(x,p,color="blue",legend="polynôme")
fig1.line(x,f(x),color='green',legend='f(x)',line_dash=[4, 4])

            # erreur
fig3 = figure(width=490, height=300,title="Erreur - répartition uniforme")
fig3.line(x,abs(p-f(x)))

fig1.legend.location = "top_left"
fig3.legend.location = "top_left"
show(row(fig1,fig3))

# ----------------------- pour répartition de Tchebychef ----------------------------------------
    # définition des vecteurs et utilisation des fonctions
xk = np.array([np.cos((2*k+1)*np.pi/(2*n+2)) for k in range(0,n+1)])
yk = f(xk)
coef = compute_divided_diff_coef(xk, yk)
p = poly_interp(coef, xk, x)

    # tracés
    
                # polynôme d'interpolation
fig2 = figure(width=490, height=300,title="Méthode des différences divisées - répartition type Tchebychev")
fig2.x(xk,yk,color="red",legend="points d'interpolation")
fig2.line(x,p,color="blue",legend="polynôme")
fig2.line(x,f(x),color='green',line_dash=[4, 4],legend="f(x)")

                # erreur
fig4 = figure(width=490, height=300,title="Erreur - répartition type Tchebychev")
fig4.line(x,abs(p-f(x)))

fig2.legend.location = "top_left"
fig4.legend.location = "top_left"
show(row(fig2,fig4))

Le phénomène de Runge est résolu en prenant une répartition de type Tchebychev dans le cas de f. L'erreur d'approximation commise est plus faible quand dans le cas d'une répartition uniforme.

Nous remarquons de plus que l'erreur diminiue avec le nombre de points d'interpolation que l'on prend. Cependant, pour n=100 par exemple, on voit que les erreurs en machine (du fait de la non symétrie des erreurs sur l'intervalle) prennent de l'importance. Les valeurs du polynôme explosent proche de 1 dans le cas d'une répartition uniforme et proche de -1 dans le cas d'une répartition de type Tchebychev.

# Interpolation de Hermite (bonus)

**Implémenter la méthode et tester sur l'exemple suivant :**

$$
f(x) =\frac{1}{0.1 +x^2}
$$

On utilisera la formule suivante pour calculer Qn(f)(x):
    $$ Qn(f)(x)= \sum_{k=0}^n l_k(x)^2 [(x-x_k)(f'(x_k)-2l'_k(x_k)f(x_k))+f(x_k)] = \sum_{k=0}^n l_k(x)^2 [(x-x_k)A_k+f(x_k)] $$ 
Cela permet de stocker les valeurs de $A_k$ et de ne pas avoir à les recalculer pour chaque $a \in X$ (vecteur contenant les abscisses des points d'évaluation du polynôme, utilisé lors du tracé).

In [7]:
# Implémenter la méthode

# calcul de Ak
def A(Xk,fX,dfX):
    L_Ak=[]     # La liste contenant les Ak
    
    for k in range(len(Xk)):
        
        l_prime=0                  # on calcul lk'(xk) pour chaque k
        for j in range(len(Xk)):
            if j!=k:
                l_prime=l_prime+1/(Xk[k]-Xk[j])  # la formule de lk' est simplifiée comme on évalue en xk 
        Ak=dfX[k]-2*l_prime*fX[k]   
        
        L_Ak.append(Ak)
    
    return(np.array(L_Ak))

# calcul de lk(a), a est un scalaire
def l(k,X,a):  
    
    lk=1   # lk(x), comme c'est un produit, on l'initialise à 1.
    
    for j in range(len(X)):
        if j!=k:
            lk=lk*(a-X[j])/(X[k]-X[j])  # on pourrait légèrement améliorer en calculant en dehors de la fonction
                                        # le produit de Newton au numérateur sachant qu'avec Pi le produit en question, 
                                        # on a : PI_(k+1)(x)=PI_(k)(x) * (x-xk)/(x-xk+1). 
    
    return lk

def hermite(Xk,X,f,df):
    
    n=len(Xk)
    fXk=f(Xk)    # on calcule une fois pour toute f(Xk) et f'(Xk)
    dfXk=df(Xk)
    
    # calcul des A_k
    LA=A(Xk,fXk,dfXk)
    
    H=[]        # contient les évaluations aux différents point du polynôme d'interpolation de Horner
    for x in X:
        
        Qnx=0   # initialisation au point x
        for k in range(n):
            lk=l(k,Xk,x)
            Qnx+=lk*lk*((x-Xk[k])*LA[k]+fXk[k]) # on applique la formule donné en introduction de cette question
        
        H.append(Qnx)
        
    return H
        

In [8]:
# on définit la fonction utilisée et sa dérivée
def f(x):
    return (1/(0.1+x*x))

def df(x):
    y=f(x)
    return (-2*x*y*y)


In [9]:
# Tracer les points (xk, yk) et le polynome d'interpolation de Hermite h, 
# En utilisant les fonctions de la partie précedente, tracer les points (xk, yk) et le polynome p 
# pour différentes valeurs de n et différents choix de xk (uniforme et Chebyshev).
# Tracer également l'erreur entre f et p.
a = -1
b = 1
x = np.linspace(a,b,1000)

n = 10

# ---------------- pour xk uniformément réparti --------------------------------------
     # définition des vecteurs et utilisation des fonctions
xk = np.linspace(a,  b, n+1)
yk = f(xk)

p = hermite(xk,x,f,df)

coef = compute_divided_diff_coef(xk, yk)
p1 = poly_interp(coef, xk, x)
    
    # tracés
    
            # polynômes d'interpolation
fig1 = figure(width=490, height=300,title="Interpolation de Hermite - répartition uniforme des abscisses")
fig1.x(xk,yk,color="red",legend="points d'interpolation")
fig1.line(x,p,color="blue",legend="polyôme")
# fig1.line(x,p1,color="black",legend="Polynôme Lagrange") # pour tracer le polynôme de Lagrange
fig1.line(x,f(x),color='green',legend='f(x)',line_dash=[4,4])

            # erreur
fig3 = figure(width=490, height=300,title="Erreur - répartition uniforme des abscisses")
fig3.line(x,abs(p-f(x)),legend="Erreur - polynôme Hermite")
fig3.line(x,abs(p1-f(x)),color="red",legend="Erreur - Polynôme Lagrange")

fig1.legend.location = "top_left"
fig3.legend.location = "top_left"
show(row(fig1,fig3))


#--------------- pour répartition de Tchebychef--------------------------------------
    # définition des vecteurs et utilisation des fonctions
xk = np.array([np.cos((2*k+1)*np.pi/(2*n+2)) for k in range(0,n+1)])
yk = f(xk)

p = hermite(xk,x,f,df)

coef = compute_divided_diff_coef(xk, yk)
p1 = poly_interp(coef, xk, x)

 # tracés
    
            # polynômes d'interpolation
        
fig2 = figure(width=490, height=300,title="Interpolation de Hermite - répartition type Tchebychev")
fig2.x(xk,yk,color="red",legend="points d'interpolation")
fig2.line(x,p,color="blue",legend="Polynôme Hermite")
# fig2.line(x,p1,color="black",legend="Polynôme Lagrange")
fig2.line(x,f(x),color='green',legend='f(x)',line_dash=[4,4])

            # erreur
fig4 = figure(width=490, height=300,title="Erreur - répartition type Tchebychev des abscisses")
fig4.line(x,abs(p-f(x)),legend="Erreur - Polynôme Hermite")
fig4.line(x,abs(p1-f(x)),color="red",legend="Erreur - Polynôme Lagrange")


fig2.legend.location = "top_left"
fig4.legend.location = "top_left"
show(row(fig2,fig4))


On remarque que pour les deux répartitions et pour n quelconque (test effectué pour un $n_{max}=100$) l'interpolation de hermite admet une erreur beaucoup moins importante que pour l'interpolation de Lagrange. Cependant:
* dans une répartion uniforme, l'erreur de Hermite diverge pour des valeurs proches de 1 et n =100 (arrondis en machine).
* dans une répartition de type Tchebychev, l'erreur de Hermite maximale vaut $6.10^{-14}$ environ pour n=100 (l'erreur maximale de Lagrange diverge). 

On pourrait pour réduire ces problèmes d'arrondis pour une répartition uniforme, travailler d'abord avec les $x_k$ proche du milieu de l'intervalle pour ensuite travailler avec les extrêmes, comme cela a été  exposé en amphi.

# Krigeage (bonus)

**On considère les 13 points $(x_k,y_k)$ suivants :**

$$(-3,0.5),\ (-2.5,0),\ (-2,2),\ (-1.5,1.5),\ (-1,4),\ (-0.5,2),\ (0,1.5),\ (0.5,3),\ (1,0),\ (1.5,2),\ (2,1),\ (2.5,2.5),\ (3,3)$$

**Implémenter $G$, $A$ et $Y$, puis résoudre le système linéaire et tracer la fonction $u$ obtenue pour $g(x)=x$ et $g(x)=x^3$. Comparer avec le polynôme d'interpolation de Lagrange. Commenter les résultats obtenus.**

In [10]:
xk = np.linspace(-3,3,13)
yk = np.array([0.5,0,2,1.5,4,2,1.5,3,0,2,1,2.5,3])
x = np.linspace(-3,3,1000)

In [11]:
def g(x):
    return x

In [12]:
A = np.zeros((13,2))
s=np.shape(A)
for k in range(s[0]):
    A[k][0]=1
    A[k][1]=xk[k]

G = np.zeros((13,13))
for i in range(len(xk)):
    for j in range(len(xk)): # on peut améliorer (G est symétrique)
        G[i][j]=g(abs(xk[j]-xk[i]))

M = np.concatenate((np.concatenate((G,np.transpose(A))),np.concatenate((A,np.zeros((2,2))))),axis=1)

Y = np.zeros(15)
for k in range(len(Y)-2):
    Y[k]=yk[k]
Y[-2],Y[-1]=0,0

# Résolution de M S = Y
S = np.linalg.solve(M,Y)

# Evaluation de la fonction de krigeage u en un point x
def u_kri(x, S, xk):
    u=[]
    for a in x:
        somme=0
        for k in range(len(xk)):
            somme+=S[k]*g(abs(a-xk[k]))        
        u.append(S[-2]+S[-1]*a+somme)
    return u

# Afficher la fonction de krigeage et comparer avec le polynome d'interpolation de Lagrange
p1=u_kri(x,S,xk)

coef = compute_divided_diff_coef(xk, yk)
p2 = poly_interp(coef, xk, x)
fig1 = figure(width=490, height=300,title="Interpolation de Krigeage")
fig1.x(xk,yk,color="red")
fig1.line(x,p1,color="blue",legend="Krigeage")
fig1.line(x,p2,color='green',legend='Lagrange')

fig3 = figure(width=490, height=300,title="Différence absolue entre les deux interpolations")
fig3.line(x,abs(p1-p2))

fig1.legend.location = "top_left"

show(row(fig1,fig3))

Pour la fonction $x^{3}$.

In [13]:
def g(x):
    return x*x*x

In [14]:
A = np.zeros((13,2))
s=np.shape(A)
for k in range(s[0]):
    A[k][0]=1
    A[k][1]=xk[k]

G = np.zeros((13,13))
for i in range(len(xk)):
    for j in range(len(xk)): # on peut améliorer (G est symétrique)
        G[i][j]=g(abs(xk[j]-xk[i]))

M = np.concatenate((np.concatenate((G,np.transpose(A))),np.concatenate((A,np.zeros((2,2))))),axis=1)

Y = np.zeros(15)
for k in range(len(Y)-2):
    Y[k]=yk[k]
Y[-2],Y[-1]=0,0

# Résolution de M S = Y
S = np.linalg.solve(M,Y)

# Evaluation de la fonction de krigeage u en un point x
def u_kri(x, S, xk):
    u=[]
    for a in x:
        somme=0
        for k in range(len(xk)):
            somme+=S[k]*g(abs(a-xk[k]))        
        u.append(S[-2]+S[-1]*a+somme)
    return u

# Afficher la fonction de krigeage et comparer avec le polynome d'interpolation de Lagrange
p1=u_kri(x,S,xk)

coef = compute_divided_diff_coef(xk, yk)
p2 = poly_interp(coef, xk, x)
fig1 = figure(width=490, height=300,title="Interpolation de Krigeage")
fig1.x(xk,yk,color="red")
fig1.line(x,p1,color="blue",legend="Krigeage")
fig1.line(x,p2,color='green',legend='Lagrange')

fig3 = figure(width=490, height=300,title="Différence absolue entre les deux interpolations")
fig3.line(x,abs(p1-p2))

fig1.legend.location = "top_left"
show(row(fig1,fig3))

Nous remarquons que l'interpolation de Lagrange diverge pour des valeurs proches de 3. L'interpolation de Lagrange est un polynôme et est donc infiniment dérivable. Chaque point d'interpolation implique donc des contraintes sur les points d'interpolation suivants qui, dans notre cas, se situent en fin d'intervalle. C'est ce phénomène qui créé les grandes variations observées par rapport à l'interpolation de Krigeage lorsqu'on approche de 3. 

Dans le cas de l'interpolation de Krigeage, ces contraintres ne sont pas présentes: on impose seulement une continuité par morceaux sur les points d'interpolation.